In [3]:
import keras

ModuleNotFoundError: No module named 'keras'

In [2]:
import os
import numpy as np
from conll_dictorizer import CoNLLDictorizer, Token, save
import datasets as ds
import sys
from sklearn.feature_extraction import DictVectorizer
import time
from keras import models, layers
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import load_model
import math
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import LSTM, Bidirectional, SimpleRNN, Dense
from keras.callbacks import ModelCheckpoint

trace = False
path = str(open('path.conf', 'r').read()).strip()

OPTIMIZER = 'rmsprop'
BATCH_SIZE = 128
EPOCHS = 1
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 150
LSTM_UNITS = 512

def load(file):
    """
    Return the embeddings in the from of a dictionary
    :param file:
    :return:
    """
    file = file
    embeddings = {}
    glove = open(file)
    for line in glove:
        values = line.strip().split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        embeddings[word] = vector
    glove.close()
    embeddings_dict = embeddings
    embedded_words = sorted(list(embeddings_dict.keys()))
    return embeddings_dict

def build_sequences(corpus_dict, key_x='form', key_y='ppos', tolower=True): #key_u='upos' -> 'ppos'
    """
    Creates sequences from a list of dictionaries
    :param corpus_dict:
    :param key_x:
    :param key_y:
    :return:
    """
    X = []
    Y = []
    for sentence in corpus_dict:
        x = []
        y = []
        for word in sentence:
            x += [word[key_x]]
            y += [word[key_y]]
        if tolower:
            x = list(map(str.lower, x))
        X += [x]
        Y += [y]
    return X, Y

def to_index(X, idx):
    """
    Convert the word lists (or POS lists) to indexes
    :param X: List of word (or POS) lists
    :param idx: word to number dictionary
    :return:
    """
    X_idx = []
    for x in X:
        # We map the unknown words to one
        x_idx = list(map(lambda x: idx.get(x, 1), x))
        X_idx += [x_idx]
    return X_idx

def predict_sentence(sentence, model, word_idx, 
                     vocabulary_words, rev_idx_pos, verbose=False):
    # Predict one sentence
    sentence = sentence.split()
    word_idxs = to_index([sentence], word_idx)
    word_idx_padded = pad_sequences(word_idxs)

    pos_idx_pred = model.predict(word_idx_padded)
    # We remove padding
    pos_idx_pred = pos_idx_pred[0][-len(sentence):]
    pos_idx = list(map(np.argmax, pos_idx_pred))
    pos = list(map(rev_idx_pos.get, pos_idx))
    if verbose:
        print('Sentence', sentence)
        print('Sentence word indexes', word_idxs)
        print('Padded sentence', word_idx_padded)
        print('POS predicted', pos_idx_pred[0])
        print('POS shape', pos_idx_pred.shape)
    return pos

ModuleNotFoundError: No module named 'keras'

In [2]:
# coding=utf-8
"""
CoNLL 2009 file readers and writers for the parts of speech.
Version with a class modeled as a vectorizer
"""
__author__ = "Pierre Nugues"

import regex as re


def save(file, corpus_dict, column_names):
    """
    Saves the corpus in a file
    :param file:
    :param corpus_dict:
    :param column_names:
    :return:
    """
    with open(file, 'w') as f_out:
        for sentence in corpus_dict:
            sentence_lst = []
            for row in sentence:
                items = map(lambda x: row.get(x, '_'), column_names)
                sentence_lst += ' '.join(items) + '\n' #was \t
            sentence_lst += '\n'
            f_out.write(''.join(sentence_lst))


class Token(dict):
    pass


class CoNLLDictorizer:

    def __init__(self, column_names, sent_sep='\n\n', col_sep=' +'):
        self.column_names = column_names
        self.sent_sep = sent_sep
        self.col_sep = col_sep

    def fit(self):
        pass

    def transform(self, corpus):
        corpus = corpus.strip()
        sentences = re.split(self.sent_sep, corpus)
        return list(map(self._split_in_words, sentences))

    def fit_transform(self, corpus):
        return self.transform(corpus)

    def _split_in_words(self, sentence):
        rows = re.split('\n', sentence)
        return [Token(dict(zip(self.column_names,
                               re.split(self.col_sep, row))))
                for row in rows]


if __name__ == '__main__':
    train_file = 'test.txt'

    column_names = ['id', 'form', 'lemma', 'cpos', 'pos', 'feats']
    train = open(train_file).read().strip()
    conll_dict = CoNLLDictorizer(column_names, col_sep='\t')
    train_dict = conll_dict.transform(train)

    print(train_dict[0])
    print(train_dict[0][0])
    print(type(train_dict[0][0]))
    print(train_dict[0][0]['form'])
    print(train_dict[1])
    tok = Token({'id': '1', 'form': 'La', 'lemma': 'el', 'cpos': 'd', 'pos': 'da', 'feats': 'num=s|gen=f'})
    print(tok['form'])
    print('form' in tok)

    save('out', train_dict, column_names)

    tok_dict = {'id': '1', 'form': 'La', 'lemma': 'el', 'cpos': 'd', 'pos': 'da', 'feats': 'num=s|gen=f'}
    tok_dict2 = {'id': '1', 'form': 'La', 'lemma': 'el', 'cpos': 'd', 'pos': 'da', 'feats': 'num=s|gen=f'}

    tok_set = set(tok_dict)
    print(tok_set)

    tok_set = tok_set.union(tok_dict2)
    print(tok_set)

    print(tok.keys())

    # exit()
    word_set = set()
    word_set = set(tok_dict.values())
    print(list(word_set))

    word_set = set()
    word_set = set(tok.values())
    print(list(word_set))

    word_set = set()
    word_set.update(tok.values())
    print(list(word_set))

    word_set = set()
    print("Token value:", tok.values())
    word_set = word_set.union(set(tok.values()))
    print(list(word_set))


ModuleNotFoundError: No module named 'regex'

In [ ]:

from conll_dictorizer import CoNLLDictorizer, Token


def load_conll2009_pos():
    train_file = '../../../corpus/conll2009/en/CoNLL2009-ST-English-train-pos.txt'
    dev_file = '../../../corpus/conll2009/en/CoNLL2009-ST-English-development-pos.txt'
    test_file = '../../../corpus/conll2009/en/CoNLL2009-ST-test-words-pos.txt'
    test2_file = 'simple_pos_test.txt'

    column_names = ['id', 'form', 'lemma', 'plemma', 'pos', 'ppos']

    train_sentences = open(train_file).read().strip()
    dev_sentences = open(dev_file).read().strip()
    test_sentences = open(test_file).read().strip()
    test2_sentences = open(test2_file).read().strip()
    return train_sentences, dev_sentences, test_sentences, column_names


path = str(open("path.conf", "r").read()).rstrip('\n')

BASE_DIR = path;


def load_conll2003_en():
    train_file = BASE_DIR + 'corpus/eng.train'
    dev_file = BASE_DIR + 'corpus/eng.valid'
    test_file = BASE_DIR + 'corpus/eng.test'
    column_names = ['form', 'ppos', 'pchunk', 'ner']
    train_sentences = open(train_file).read().strip()
    dev_sentences = open(dev_file).read().strip()
    test_sentences = open(test_file).read().strip()
    return train_sentences, dev_sentences, test_sentences, column_names


if __name__ == '__main__':
    train_sentences, dev_sentences, test_sentences, column_names = load_conll2003_en()

    conll_dict = CoNLLDictorizer(column_names, col_sep=' +')
    train_dict = conll_dict.transform(train_sentences)
    print(train_dict[0])
    print(train_dict[1])


In [ ]:

train_sentences, dev_sentences, test_sentences, column_names = ds.load_conll2003_en()
conll_dict = CoNLLDictorizer(column_names)
train_dict = conll_dict.transform(train_sentences)
dev_dict = conll_dict.transform(dev_sentences)
test_dict = conll_dict.transform(test_sentences)

#overwrite column names to include our prediction column
column_names.append("pred")

#change column names to have our predition column

embeddings_file = os.path.join(path, "corpus/glove.6B.100d.txt")
embeddings_dict = load(embeddings_file)

X_train_cat, Y_train_cat = build_sequences(train_dict) # Xfor words, Y for parts of speech (pos)

vocabulary_words = sorted(list( set([word for sentence in X_train_cat for word in sentence])))
pos = sorted(list(set([pos for sentence in Y_train_cat for pos in sentence])))

NB_CLASSES = len(pos)

embeddings_words = embeddings_dict.keys()
if(trace):
    print('Words in GloVe:',  len(embeddings_dict.keys()))
vocabulary_words = sorted(list(set(vocabulary_words + list(embeddings_words))))
cnt_uniq = len(vocabulary_words) + 2
if(trace):
    print('# unique words in the vocabulary: embeddings and corpus:', cnt_uniq)

# We start at one to make provision for the padding symbol 0 
# in RNN and LSTMs and 1 for the unknown words
rev_word_idx = dict(enumerate(vocabulary_words, start=2))
rev_pos_idx = dict(enumerate(pos, start=2))
word_idx = {v: k for k, v in rev_word_idx.items()}
pos_idx = {v: k for k, v in rev_pos_idx.items()}
print('word index:', list(word_idx.items())[:10])
print('POS index:', list(pos_idx.items())[:10])

# We create the parallel sequences of indexes
X_idx = to_index(X_train_cat, word_idx)
Y_idx = to_index(Y_train_cat, pos_idx)
print('First sentences, word indices', X_idx[:3])
print('First sentences, POS indices', Y_idx[:3])

X = pad_sequences(X_idx)
Y = pad_sequences(Y_idx)

print(X[0])
print(Y[0])

# The number of POS classes and 0 (padding symbol)
Y_train = to_categorical(Y, num_classes=len(pos) + 2)
print(Y_train[0])

rdstate = np.random.RandomState(1234567)
embedding_matrix = rdstate.uniform(-0.05, 0.05, (len(vocabulary_words) + 2, EMBEDDING_DIM))
for word in vocabulary_words:
    if word in embeddings_dict:
        # If the words are in the embeddings, we fill them with a value
        embedding_matrix[word_idx[word]] = embeddings_dict[word]
if(trace):
    print('Shape of embedding matrix:', embedding_matrix.shape)
    print('Embedding of table', embedding_matrix[word_idx['table']])
    print('Embedding of the padding symbol, idx 0, random numbers', embedding_matrix[0])

model = models.Sequential()
model.add(layers.Embedding(len(vocabulary_words) + 2, EMBEDDING_DIM, mask_zero=True, input_length=None))
model.layers[0].set_weights([embedding_matrix])
# The default is True
model.layers[0].trainable = False
#model.add(LSTM(100, return_sequences=True))
#model.add(Bidirectional(SimpleRNN(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100, dropout = 0.25, recurrent_dropout = 0.25, return_sequences=True)))
model.add(layers.Dropout(0.25))
model.add(Dense(NB_CLASSES + 2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.summary()
model.fit(X, Y_train, epochs=EPOCHS, batch_size=32, callbacks = [ModelCheckpoint("bestout.h5", save_best_only = True)]) #sök på model checkpoint
#lägg till validation_data tuple (x, y)
#dev_dict = validation
#remeber to load model!!

In [ ]:
# In X_dict, we replace the words with their index
X_test_cat, Y_test_cat = build_sequences(test_dict)
# We create the parallel sequences of indexes
X_test_idx = to_index(X_test_cat, word_idx)
Y_test_idx = to_index(Y_test_cat, pos_idx)

if(trace):
    print('X[0] test idx', X_test_idx[0])
    print('Y[0] test idx', Y_test_idx[0])

X_test_padded = pad_sequences(X_test_idx)
Y_test_padded = pad_sequences(Y_test_idx)
if(trace):
    print('X[0] test idx passed', X_test_padded[0])
    print('Y[0] test idx padded', Y_test_padded[0])
# One extra symbol for 0 (padding)
Y_test_padded_vectorized = to_categorical(Y_test_padded, 
                                          num_classes=len(pos) + 2)
if(trace):
    print('Y[0] test idx padded vectorized', Y_test_padded_vectorized[0])
    print(X_test_padded.shape)
    print(Y_test_padded_vectorized.shape)

# Evaluates with the padding symbol
test_loss, test_acc = model.evaluate(X_test_padded, 
                                     Y_test_padded_vectorized)
if(trace):
    print('Loss:', test_loss)
    print('Accuracy:', test_acc)

corpus_pos_predictions = model.predict(X_test_padded)
pos_pred_num = []
for sent_nbr, sent_pos_predictions in enumerate(corpus_pos_predictions):
    pos_pred_num += [sent_pos_predictions[-len(X_test_cat[sent_nbr]):]]
print(pos_pred_num[:2])

#add pos pred to test_dict
pos_pred = []
for sentence in pos_pred_num:
    pos_pred_idx = list(map(np.argmax, sentence))
    pos_pred_cat = list(map(rev_pos_idx.get, pos_pred_idx))
    pos_pred += [pos_pred_cat]
    i=0
    for x in pos_pred_idx:
       # test_dict[x].append('pred')
        test_dict[x].append(pos_pred_cat[i])
        i=i+1


print(pos_pred[:2])
print(Y_test_cat[:2])

total, correct, total_ukn, correct_ukn = 0, 0, 0, 0
for id_s, sentence in enumerate(X_test_cat):
    for id_w, word in enumerate(sentence):
        total += 1
        if pos_pred[id_s][id_w] == Y_test_cat[id_s][id_w]:
            correct += 1
        # The word is not in the dictionary
        if word not in word_idx:
            total_ukn += 1
            if pos_pred[id_s][id_w] == Y_test_cat[id_s][id_w]:
                correct_ukn += 1

print('total %d, correct %d, accuracy %f' % 
      (total, correct, correct / total))
if total_ukn != 0:
    print('total unknown %d, correct %d, accuracy %f' % 
          (total_ukn, correct_ukn, correct_ukn / total_ukn))

sentences = ["That round table might collapse .", "The man can learn well .", "The man can swim .", "The man can simwo ."]
for sentence in sentences:
    y_test_pred_cat = predict_sentence(sentence.lower(), model, word_idx, vocabulary_words, rev_pos_idx, trace)
    print(sentence)
    print(y_test_pred_cat)

save('trainout.txt', test_dict, column_names)